# T2.8 Extraer con selenium datos de licitacións do Concello da Coruña
## Ralphy Núñez Mercado

Queremos facer un código que garde nunha BBDD a Microsoft SQL Server a información dos expedientes de licitación do Concello da Coruña e nun directorio (no actual) as capturas de pantalla de cada expediente.

Para montar o Microsoft SQL Server sigue a guía de: https://jfsanchez.es/docs/docker-5-mssql-server/

Debes empregar selenium e a páxina: https://contrataciondelestado.es

Debes navegar por ela coma se indica no documento.

Por cada elemento (expediente), meterse nel e facer captura de pantalla (automáticamente, coa API de selenium).

Entrega:

O código nun jupyter notebook.
A BBDD exportada de Microsoft SQL Server.
Un zip coas capturas de pantalla que fixo selenium.

### ⬇️ Instalar librerias necesarias

In [22]:
!conda install pip -y || true
!conda install -c conda-forge selenium -y || true
!pip install webdriver_manager || true
!conda install pandas -y || true
!conda install sqlalchemy -y || true

Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\ralph\.conda\envs\IAbigdata

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    selenium-4.31.0            |     pyh29332c3_0         310 KB  conda-forge
    selenium-manager-4.31.0    |       ha073cba_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following packages will be UPDATED:

  selenium                              4.29.0-pyh29332c3_0 --> 4.31.0-pyh29332c3_0 
  selenium-manager                        4.2

### 🦎 Instalar GeckoDriver

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

### ⬇️ Imports necesarios.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import itertools
from selenium.common.exceptions import NoSuchElementException
import json
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy import insert
import pyodbc
import pandas as pd

### ✏️ Definir Funciones

In [ ]:
def select_opcion_portal(driver,link):
    driver.find_element(By.XPATH, f"//div/p/a[contains(@href,'{link}')]").click()

def click_opciones_busqueda(driver,opcion):
    driver.find_element(By.XPATH, f"//div/a/p/img[contains(@id,'{opcion}')]").click()
    
def buscar_by_id(driver,id):
    driver.find_element(By.ID,id).click()

def buscar_elementos_lista(driver, elemento, n=None):
    if n is not None:
        driver.find_element(By.XPATH, f"(//td[@class='tafelTreecanevas']/div[text()='A Coruña'])[{n}]").click()
    else:
        driver.find_element(By.XPATH, f"//td[@class='multiline'][following-sibling::td[2]/div[text()='{elemento}']]").click()

def select_combo_organo(driver,combo,valor):
    driver.find_element(By.XPATH, f"//select[contains(@id,'{combo}')]//option[@value='{valor}']").click()

def getDatos(driver, XPath):
    # Buscar todos los elementos que coinciden con el XPath
    elementos = driver.find_elements(By.XPATH, XPath)

    dic = {}
    for elemento in elementos:
        # Dentro de cada elemento, buscar todos los elementos li
        lista = elemento.find_elements(By.TAG_NAME, 'li')

        if len(lista) > 1:
            # Si hay al menos dos li:
            # lista[0] es la clave 
            # lista[1] es el valor
            dic[lista[0].text] = " ".join(lista[1].text.split())
        else:
            # Si solo hay un li, es decir, no hay valor guarda el valor como None
            dic[lista[0].text] = None

    return dic 

def limpiar_datos_json(json_data):
    datos_limpios = []

    for elemento in json_data:
        nuevo_elemento = {}

        for key, valor in elemento.items():
            # Quitar ":" al final de las claves
            clave_limpia = key.rstrip(":")

            # Si el valor es "Ver detalle de la adjudicación", cambiarlo a None
            if valor == "Ver detalle de la adjudicación":
                nuevo_elemento[clave_limpia] = None

            # Convertir valores vacíos a None
            elif valor == "":
                nuevo_elemento[clave_limpia] = None

            # Convertir valores con Euros a float
            elif isinstance(valor, str) and "Euros" in valor:
                # Limpiar la cadena y convertir el valor a float
                numeric_str = valor.replace("Euros", "").replace(".", "").replace(",", ".").strip()
                try:
                    nuevo_elemento[clave_limpia] = float(numeric_str)
                except ValueError:
                    nuevo_elemento[clave_limpia] = None

            # Convertir "ID del Órgano de Contratación" a int
            elif clave_limpia == "ID del Órgano de Contratación" and isinstance(valor, str) and valor.isdigit():
                nuevo_elemento[clave_limpia] = int(valor)

            else:
                nuevo_elemento[clave_limpia] = valor

        datos_limpios.append(nuevo_elemento)

    return datos_limpios


def obtenerLinks(driver,nombre_json,path_links,pagina_siguiente,n_paginas):
    for i in range(1, n_paginas + 1): # iterar sobre las páginas
        path_links = driver.find_elements(By.XPATH,path_links)

        links.append([item.get_attribute("href") for item in path_links if item.get_attribute("href")]) # Obtener el atributo href de cada elemento 

        try: # pasar a la siguiente página
            buscar_by_id(driver, pagina_siguiente)
            time.sleep(0.1)  # test
        except NoSuchElementException:
            print("Última página.")
            break  

    links = list(itertools.chain(*links))
    with open(nombre_json, "w") as f:
        json.dump(links, f, indent=4)

def convertir_fecha(valor):
    if valor:
        try:
            # Intentar convertir el valor a una fecha con el formato esperado
            return datetime.strptime(valor, "%d/%m/%Y %H:%M")
        except ValueError:
            # Si no se puede convertir, retornar None (en lugar de texto no válido)
            return None
    return None  # Si el valor es vacío o nulo, retornamos None


### Definir Variables

In [25]:

URL = "https://contrataciondelestado.es/wps/portal/plataforma"

lista_tree = ['ENTIDADES LOCALES','Galicia','A Coruña','Ayuntamientos','A Coruña']

links = []

publicaciones = '/wps/portal/licitaciones'

licitaciones = 'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:logoFormularioBusqueda'

busqueda_avanzada = 'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textBusquedaAvanzada'

organizacion_contrante = 'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:idSeleccionarOCLink'

opcion_combo = 'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:comboNombreOrgano'

valor_combo = '93655277'

btn_añadir = 'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:botonAnadirMostrarPopUpArbolEO'

btn_buscar = 'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:button1'

total_pag = '//*[@id="viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textfooterInfoTotalPaginaMAQ"]'

In [ ]:
driver = webdriver.Firefox()

driver.get(URL)


select_opcion_portal(driver,publicaciones) # click en publicaciones

click_opciones_busqueda(driver,licitaciones) # click en licitaciones

buscar_by_id(driver,busqueda_avanzada) # click en busqueda avanzada

buscar_by_id(driver,organizacion_contrante) # click en selecionar (organizacion contratante)


for i, elemento in enumerate(lista_tree): # iterar sobre la lista para buscar los datos que queremos obtener
    if i == len(lista_tree) - 1:
        buscar_elementos_lista(driver, elemento, 2)
    else:
        buscar_elementos_lista(driver, elemento)



select_combo_organo(driver,opcion_combo,valor_combo) # selecionar opcion combo organo

buscar_by_id(driver,btn_añadir) # click en boton añadir
buscar_by_id(driver,btn_buscar) # click en buscar

time.sleep(0.2) # test

n_paginas = int(driver.find_element(By.XPATH,total_pag).text)

obtenerLinks(driver,'links.json',"//table[contains(@id,'myTablaBusquedaCustom')]//a[2]",'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:footerSiguiente',n_paginas)


driver.close()

55
Última página.


### 🔄️ Iterar sobre los links obtenidos y crear un json con los datos de cada página de licitaciones además de sacar capturas de cada página.

In [ ]:
driver = webdriver.Firefox()


# Cargar los enlaces desde el archivo JSON
with open("links.json", "r") as f:
    links = json.load(f)

lista_jsons = []

for i, link in enumerate(links):
    driver.get(link)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    # Tomar captura de la página completa 
    screenshot = driver.get_full_page_screenshot_as_png() 
    with open(f'capturas/{i}.png', 'wb') as file:
        file.write(screenshot)

    # Obtener datos de cada página 
    datos =  getDatos(driver,"//ul[@class='altoDetalleLicitacion']")
    datos.update(getDatos(driver,"//ul[@class='ancho100 altoDetalleLicitacion']"))
    lista_jsons.append(datos)

# Guardar los datos en un json
with open("resultados.json", "w", encoding="utf-8") as f:
    json.dump(lista_jsons, f, ensure_ascii=False, indent=4)

driver.quit()

### ⬇️ Cargar el json con los datos obtenidos.

In [26]:
with open("resultados.json", "r") as f:
    resultados = json.load(f)

### ✅ Comprobar el tamaño del json para saber si hemos obtenido los datos correctamente.

In [27]:
print(len(resultados))

1082


### 🧹 Limpiar los datos para cambiar algunos formatos de los datos.

In [28]:
resultados = limpiar_datos_json(resultados)

with open("resultados.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

### 📋 Datos de conexión

In [7]:
db_host = "10.133.28.46" # ip cesga
db_port=41433 # Puerto Microsoft SQL Server
db_user = "sa" # Usuario Admin por defecto
db_passwd="Abc12300" # Contraseña 
db_name="licitaciones" # Nombre de la BD

### ✏️ Definición de la connectionString y creación del engine.

In [8]:
connectionString = f'mssql+pyodbc://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}?driver=ODBC+Driver+18+for+SQL+Server&Encrypt=no'
engine = create_engine(connectionString)

#### Para poder hacer uso de nuestro engine tendremos que instalar la libreria para Microsft SQL Server y el driver de este mismo
#### Librería: ```python !pip install pyodbc ```
#### Driver: https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16


### ⛏️ Creacion de la tabla desde python


```sql
CREATE TABLE Licitaciones (
    ID_Pub_TED VARCHAR(512) NULL,
    Organo_Contratacion VARCHAR(512) NULL,
    ID_Organo_Contratacion FLOAT NULL,
    Estado_Licitacion VARCHAR(256) NULL,
    Objeto_Contrato VARCHAR(256) NULL,
    Financiacion_UE VARCHAR(256) NULL,
    Presupuesto_Base_Licitacion_Sin_Impuestos FLOAT NULL,
    Valor_Est_Contrato FLOAT NULL,
    Tipo_Contrato VARCHAR(256) NULL,
    Cod_CPV TEXT NULL,
    Lugar_Ejecucion VARCHAR(256) NULL,
    Sistema_Contratacion VARCHAR(256) NULL,
    Procedimiento_Contratacion VARCHAR(256) NULL,
    Tipo_Tramitacion VARCHAR(256) NULL,
    Metodo_Presentacion_Oferta VARCHAR(256) NULL,
    Fecha_Fin_Presentacion_Oferta DATETIME NULL,
    Resultado VARCHAR(256) NULL,
    Adjudicatario VARCHAR(256) NULL,
    N_Licitadores_Presentados VARCHAR(256) NULL,
    Importe_Adjudiacion FLOAT NULL,
    Enlace_Licitacion VARCHAR(256) NULL,
    Fecha_Fin_Presentacion_Solicitud DATETIME NULL
);
```

In [30]:
create_table = "CREATE TABLE Licitaciones (ID_Pub_TED VARCHAR(512) NULL," \
                                        " Organo_Contratacion VARCHAR(512) NULL, ID_Organo_Contratacion FLOAT NULL," \
                                        " Estado_Licitacion VARCHAR(256) NULL, " \
                                        "Objeto_Contrato TEXT NULL, " \
                                        "Financiacion_UE VARCHAR(256) NULL, " \
                                        "Presupuesto_Base_Licitacion_Sin_Impuestos FLOAT NULL, " \
                                        "Valor_Est_Contrato FLOAT NULL, " \
                                        "Tipo_Contrato VARCHAR(256) NULL, " \
                                        "Cod_CPV TEXT NULL, " \
                                        "Lugar_Ejecucion VARCHAR(256) NULL, " \
                                        "Sistema_Contratacion VARCHAR(256) NULL," \
                                        " Procedimiento_Contratacion VARCHAR(256) NULL, " \
                                        "Tipo_Tramitacion VARCHAR(256) NULL," \
                                        " Metodo_Presentacion_Oferta VARCHAR(256) NULL," \
                                        "Fecha_Fin_Presentacion_Oferta DATETIME NULL, " \
                                        "Resultado VARCHAR(256) NULL," \
                                        " Adjudicatario VARCHAR(256) NULL, " \
                                        "N_Licitadores_Presentados VARCHAR(256) NULL, " \
                                        "Importe_Adjudiacion VARCHAR(256) NULL, " \
                                        "Enlace_Licitacion VARCHAR(256) NULL, " \
                                        "Fecha_Fin_Presentacion_Solicitud DATETIME NULL);"

with engine.connect() as conn:
    conn.execute(text(create_table))
    conn.commit()




### ✅ Comprobar que que la tabla se creado correctamente 

In [17]:
selectSQL = "SELECT * FROM licitaciones"
dfPanda = pd.read_sql(selectSQL, engine)
print(dfPanda.head())

Empty DataFrame
Columns: [ID_Pub_TED, Organo_Contratacion, ID_Organo_Contratacion, Estado_Licitacion, Objeto_Contrato, Financiacion_UE, Presupuesto_Base_Licitacion_Sin_Impuestos, Valor_Est_Contrato, Tipo_Contrato, Cod_CPV, Lugar_Ejecucion, Sistema_Contratacion, Procedimiento_Contratacion, Tipo_Tramitacion, Metodo_Presentacion_Oferta, Fecha_Fin_Presentacion_Oferta, Resultado, Adjudicatario, N_Licitadores_Presentados, Importe_Adjudiacion, Enlace_Licitacion, Fecha_Fin_Presentacion_Solicitud]
Index: []

[0 rows x 22 columns]


### ⬇️ Insertar los datos del json a nuestra BD

In [31]:
from sqlalchemy import text

sql = text("""
    INSERT INTO Licitaciones (
        ID_Pub_TED, Organo_Contratacion, ID_Organo_Contratacion, Estado_Licitacion, 
        Objeto_Contrato, Financiacion_UE, Presupuesto_Base_Licitacion_Sin_Impuestos, 
        Valor_Est_Contrato, Tipo_Contrato, Cod_CPV, Lugar_Ejecucion, Sistema_Contratacion, 
        Procedimiento_Contratacion, Tipo_Tramitacion, Metodo_Presentacion_Oferta, 
        Fecha_Fin_Presentacion_Oferta, Resultado, Adjudicatario, N_Licitadores_Presentados, 
        Importe_Adjudiacion, Enlace_Licitacion, Fecha_Fin_Presentacion_Solicitud
    ) VALUES (
        :ID_Pub_TED, :Organo_Contratacion, :ID_Organo_Contratacion, :Estado_Licitacion,
        :Objeto_Contrato, :Financiacion_UE, :Presupuesto_Base_Licitacion_Sin_Impuestos,
        :Valor_Est_Contrato, :Tipo_Contrato, :Cod_CPV, :Lugar_Ejecucion, :Sistema_Contratacion,
        :Procedimiento_Contratacion, :Tipo_Tramitacion, :Metodo_Presentacion_Oferta,
        :Fecha_Fin_Presentacion_Oferta, :Resultado, :Adjudicatario, :N_Licitadores_Presentados,
        :Importe_Adjudiacion, :Enlace_Licitacion, :Fecha_Fin_Presentacion_Solicitud
    )
""")

with engine.connect() as conn:
    for licitacion in resultados:
        data = {
            'ID_Pub_TED': licitacion.get('ID de publicación en TED'),
            'Organo_Contratacion': licitacion.get('Órgano de Contratación'),
            'ID_Organo_Contratacion': licitacion.get('ID del Órgano de Contratación'),
            'Estado_Licitacion': licitacion.get('Estado de la Licitación'),
            'Objeto_Contrato': licitacion.get('Objeto del contrato'),
            'Financiacion_UE': licitacion.get('Financiación UE'),
            'Presupuesto_Base_Licitacion_Sin_Impuestos': licitacion.get('Presupuesto base de licitación sin impuestos'),
            'Valor_Est_Contrato': licitacion.get('Valor estimado del contrato'),
            'Tipo_Contrato': licitacion.get('Tipo de Contrato'),
            'Cod_CPV': licitacion.get('Código CPV'),
            'Lugar_Ejecucion': licitacion.get('Lugar de Ejecución'),
            'Sistema_Contratacion': licitacion.get('Sistema de contratación'),
            'Procedimiento_Contratacion': licitacion.get('Procedimiento de contratación'),
            'Tipo_Tramitacion': licitacion.get('Tipo de tramitación'),
            'Metodo_Presentacion_Oferta': licitacion.get('Método de presentación de la oferta'),
            'Fecha_Fin_Presentacion_Oferta': convertir_fecha(licitacion.get('Fecha fin de presentación de oferta')),
            'Resultado': licitacion.get('Resultado'),
            'Adjudicatario': licitacion.get('Adjudicatario'),
            'N_Licitadores_Presentados': licitacion.get('Nº de Licitadores Presentados'),
            'Importe_Adjudiacion': licitacion.get('Importe de Adjudicación'),
            'Enlace_Licitacion': licitacion.get('Enlace a la licitación'),
            'Fecha_Fin_Presentacion_Solicitud': convertir_fecha(licitacion.get('Fecha fin de presentación de solicitud'))
        }

        conn.execute(sql, data)
    conn.commit()


# Poner WebDriverWait en todos lados 